<a href="https://colab.research.google.com/github/Olgasc/Colab-SPKPrime/blob/main/Server_inout_online_GColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Colab - Spike Prime**

Using *Colab's Virtual Machine* to save data from Spike Prime and send python files to run on the Spike Prime brick

**1.- Installing flask, pyngrok and flask-ngrok packages**

---


Installing flask, pyngrok and flask-ngrok packages to create a server accessible from internet in https://XXXX.ngrok.io




In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask-ngrok

***`Why have we installed flask-ngrok?`***

The reason is that the flask server creates a server that runs locally on the allocated runtime on google colab as localhost. In order to expose the server to the outside traffic or to make the server accessible outside the runtime globally on HTTP, we use ngrok here and since we are working with flask it is good to use flask-ngrok module of python.(https://github.com/SAH-UJA/Flask_on_colab/blob/main/Flask_and_FastAPI_on_colab.ipynb)

## Install package "***flask-jsonpify***"

To send data between Flask and Javascript using AJAX, we need to use "**JsonP**" data format to avoid CORS issues due security policy about cross-origins executions.

In this case, when we send a file or any data from Flask to Javascript we need to convert it to **JsonP** format.

"flask-jsonpify" allows to convert a python file to jsonp. Inside flask application code we will call "*from flask_jsonpify import jsonify*", and in the "return" we will convert the output (python file) to jsonp type.

In [ ]:
!pip install flask-jsonpify

## Upload python files to execute on Spike Prime

For calling a python file to be executed on the Spike Prime, we have to upload the files on the path "*content/files/*":

1. Create under "*content*" the folder "*files*"
2. Upload the python files

**The path "content/files/" can be modified. You have to be sure to change the path from retrieving the python file to return on the flask route (@app.route('/file/'))

**2.-  Create Flask server** with a public IP address on ngrok.io

---



This means ngrok is exposing our web app onto the temporary URL http://XXXXXXX.ngrok.io (it is different every time it is run). If we go to this URL we get our home page: "*Connected*".

In "/data" the server save the data from Spike Prime in a .csv file in Colab Virtual Machine (*"/content/test.csv"*) temporaly.


In [ ]:
from flask import Flask, request, send_file, json
from flask_ngrok import run_with_ngrok
import csv
from flask_jsonpify import jsonify

app = Flask(__name__)

run_with_ngrok(app)   

@app.route("/")
def home():
    return "<h1>Connected</h1>"

@app.route("/data", methods=['GET', 'POST'])
def data():
    data = request.args.get("Sensor_data")
    time = request.args.get("Timestamp")
    if (time=='0'):
      print("***Starting receiving data****")
      with open('/content/test.csv', mode='w') as csvfile:
        fieldnames= ['Timestamp', 'sensor_data']
        writer=csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer1 = csv.writer(csvfile)
        writer1.writerow([time, data])

        print({'timestamp':time, 'Sensor_data':data})
    elif (time == '-1'):
        print("End receving data")
    else:
      with open('/content/test.csv', mode='a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([time, data])
        print({'timestamp':time, 'Sensor_data':data})
    return (data)

@app.route("/file/<namefile>", methods=['GET'])
def return_file(namefile):
  print(namefile)
  file=namefile+'.py'
  with open('/content/files/'+file, mode='r') as f:
    content=f.read()
  return jsonify(content)

app.run()

 **3.- Read data sensor from file**

---



In [ ]:
import pandas
df = pandas.read_csv('/content/test.csv')
print(df)

 **4.- Plot Data Sensor**

---



In [ ]:
import matplotlib.pyplot as plt
import csv
  
x = []
y = []
  
with open('/content/test.csv', 'r') as csvfile:
    reader=csv.reader(csvfile,delimiter=',')
    next(reader)
    for row in reader:
    
      x.append(row[0])
      y.append(int(row[1]))
  
plt.plot(x, y, color = 'g', linestyle = 'dashed',
         marker = 'o',label = "Data Sensor")
  
plt.xticks(rotation = 25)
plt.xlabel('Time')
plt.ylabel('Data_Sensor')
plt.title('Sensor Report', fontsize = 20)
plt.grid()
plt.legend()
plt.show()

## Save Sensor data file .csv as a Google Sheet into our Google account

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Create our spreadsheet
sh = gc.create('Data Sensor')

# Read CSV file contents
content = open('/content/test.csv', 'r').read()

#Imports data into the first page of the spreadsheet.
#This method removes all other worksheets and then entirely replaces the contents of the first worksheet.
gc.import_csv(sh.id, content)


## Downloading data from a sheet into Python as a Pandas DataFrame

Read back the random data that we inserted above and convert the result into a Pandas DataFrame.



In [ ]:
worksheet = gc.open('Data Sensor').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)